In [1]:
import bisect
import datetime
import random

import pandas as pd
import gym
import numpy as np
import plotly.figure_factory as ff
from pathlib import Path

/tmp/ipykernel_23669/702424910.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
instance_path ='Data//ta01'

In [3]:
jobs = 0
machines = 0
instance_matrix = None
jobs_length = None
max_time_op = 0
max_time_jobs = 0
nb_legal_actions = 0
nb_machine_legal = 0
solution = None
last_solution = None
last_time_step = float("inf")
current_time_step = float("inf")
next_time_step = list()
next_jobs = list()
legal_actions = None
time_until_available_machine = None
time_until_finish_current_op_jobs = None
todo_time_step_job = None
total_perform_op_time_jobs = None
needed_machine_jobs = None
total_idle_time_jobs = None
idle_time_jobs_last_op = None
state = None
illegal_actions = None
action_illegal_no_op = None
machine_legal = None
start_timestamp = datetime.datetime.now().timestamp()
sum_op = 0

In [4]:
with open(instance_path, "r") as instance_file:
            for line_cnt, line_str in enumerate(instance_file, start=1):
                split_data = list(map(int, line_str.split()))

                if line_cnt == 1:
                    jobs, machines = split_data
                    instance_matrix = np.zeros((jobs, machines), dtype=(int, 2))
                    jobs_length = np.zeros(jobs, dtype=int)
                else:
                    assert len(split_data) % 2 == 0 and len(split_data) // 2 == machines
                    job_nb = line_cnt - 2
                    for i in range(0, len(split_data), 2):
                        machine, time = split_data[i], split_data[i + 1]
                        instance_matrix[job_nb][i // 2] = (machine, time)
                        max_time_op = max(max_time_op, time)
                        jobs_length[job_nb] += time
                        sum_op += time
max_time_jobs = max(jobs_length)



In [5]:
action_space = gym.spaces.Discrete(jobs + 1)
colors = [tuple([random.random() for _ in range(3)]) for _ in range(machines)]

observation_space = gym.spaces.Dict(
            {
                "action_mask": gym.spaces.Box(0, 1, shape=(jobs + 1,)),
                "real_obs": gym.spaces.Box(
                    low=0.0, high=1.0, shape=(jobs, 7), dtype=float
                ),
            }
        )

In [6]:
def _get_current_state_representation():
        state[:, 0] = legal_actions[:-1]
        return {
            "real_obs": state,
            "action_mask": legal_actions,
        }


In [7]:
def get_legal_actions(self):
        return legal_actions

In [8]:
def reset():
    current_time_step = 0
    next_time_step = list()
    next_jobs = list()
    nb_legal_actions = jobs
    nb_machine_legal = 0
    legal_actions = np.ones(jobs + 1, dtype=bool)
    legal_actions[jobs] = False
    solution = np.full((jobs, machines), -1, dtype=int)
    time_until_available_machine = np.zeros(machines, dtype=int)
    time_until_finish_current_op_jobs = np.zeros(jobs, dtype=int)
    todo_time_step_job = np.zeros(jobs, dtype=int)
    total_perform_op_time_jobs = np.zeros(jobs, dtype=int)
    needed_machine_jobs = np.zeros(jobs, dtype=int)
    total_idle_time_jobs = np.zeros(jobs, dtype=int)
    idle_time_jobs_last_op = np.zeros(jobs, dtype=int)
    illegal_actions = np.zeros((machines, jobs), dtype=bool)
    action_illegal_no_op = np.zeros(jobs, dtype=bool)
    machine_legal = np.zeros(machines, dtype=bool)
    for job in range(jobs):
        needed_machine = instance_matrix[job][0][0]
        needed_machine_jobs[job] = needed_machine
        if not machine_legal[needed_machine]:
            machine_legal[needed_machine] = True
            nb_machine_legal += 1
    state = np.zeros((jobs, 7), dtype=float)
    return _get_current_state_representation()


In [9]:
def _prioritization_non_final():
    if nb_machine_legal >= 1:
        for machine in range(machines):
            if machine_legal[machine]:
                final_job = list()
                non_final_job = list()
                min_non_final = float("inf")
                for job in range(jobs):
                    if (needed_machine_jobs[job] == machine and legal_actions[job]):
                        if todo_time_step_job[job] == (machines - 1):
                            final_job.append(job)
                        else:
                            current_time_step_non_final = todo_time_step_job[job]
                            time_needed_legal = instance_matrix[job][current_time_step_non_final][1]
                            machine_needed_nextstep = instance_matrix[job][current_time_step_non_final + 1][0]
                            if (time_until_available_machine[machine_needed_nextstep]== 0):
                                min_non_final = min(min_non_final, time_needed_legal)
                                non_final_job.append(job)
                    if len(non_final_job) > 0:
                        for job in final_job:
                            current_time_step_final = todo_time_step_job[job]
                            time_needed_legal = instance_matrix[job][current_time_step_final][1]
                            if time_needed_legal > min_non_final:
                                legal_actions[job] = False
                                nb_legal_actions -= 1

In [10]:
def _check_no_op():
        legal_actions[jobs] = False
        if (len(next_time_step) > 0 and nb_machine_legal <= 3 and nb_legal_actions <= 4):
            machine_next = set()
            next_time_step = next_time_step[0]
            max_horizon = current_time_step
            max_horizon_machine = [current_time_step + max_time_op for _ in range(machines)]
            for job in range(jobs):
                if legal_actions[job]:
                    time_step = todo_time_step_job[job]
                    machine_needed = instance_matrix[job][time_step][0]
                    time_needed = instance_matrix[job][time_step][1]
                    end_job = current_time_step + time_needed
                    if end_job < next_time_step:
                        return 
                    max_horizon_machine[machine_needed] = min(max_horizon_machine[machine_needed], end_job)
                    max_horizon = max(max_horizon, max_horizon_machine[machine_needed])
            for job in range(jobs):
                if not legal_actions[job]:
                    if (time_until_finish_current_op_jobs[job] > 0 and todo_time_step_job[job] + 1 < machines):
                        time_step = todo_time_step_job[job] + 1
                        time_needed = (current_time_step + time_until_finish_current_op_jobs[job])
                        while (time_step < machines - 1 and max_horizon > time_needed):
                            machine_needed = instance_matrix[job][time_step][0]
                            if (max_horizon_machine[machine_needed] > time_needed and machine_legal[machine_needed]):
                                machine_next.add(machine_needed)
                                if len(machine_next) == nb_machine_legal:
                                    legal_actions[jobs] = True
                                    return
                            time_needed += instance_matrix[job][time_step][1]
                            time_step += 1
                    elif (
                        not action_illegal_no_op[job]
                        and todo_time_step_job[job] < machines
                    ):
                        time_step = todo_time_step_job[job]
                        machine_needed = instance_matrix[job][time_step][0]
                        time_needed = (
                            current_time_step
                            + time_until_available_machine[machine_needed]
                        )
                        while (
                            time_step < machines - 1 and max_horizon > time_needed
                        ):
                            machine_needed = instance_matrix[job][time_step][0]
                            if (
                                max_horizon_machine[machine_needed] > time_needed
                                and machine_legal[machine_needed]
                            ):
                                machine_next.add(machine_needed)
                                if len(machine_next) == nb_machine_legal:
                                    legal_actions[jobs] = True
                                    return
                            time_needed += instance_matrix[job][time_step][1]
                            time_step += 1

In [11]:
def step(action: int):
        reward = 0.0
        if action == jobs:
            nb_machine_legal = 0
            nb_legal_actions = 0
            for job in range(jobs):
                if legal_actions[job]:
                    legal_actions[job] = False
                    needed_machine = needed_machine_jobs[job]
                    machine_legal[needed_machine] = False
                    illegal_actions[needed_machine][job] = True
                    action_illegal_no_op[job] = True
            while nb_machine_legal == 0:
                reward -= increase_time_step()
            scaled_reward = _reward_scaler(reward)
            _prioritization_non_final()
            _check_no_op()
            return (
                _get_current_state_representation(),
                scaled_reward,
                _is_done(),
                {},
            )
        else:
            current_time_step_job = todo_time_step_job[action]
            machine_needed = needed_machine_jobs[action]
            time_needed = instance_matrix[action][current_time_step_job][1]
            reward += time_needed
            time_until_available_machine[machine_needed] = time_needed
            time_until_finish_current_op_jobs[action] = time_needed
            state[action][1] = time_needed / max_time_op
            to_add_time_step = current_time_step + time_needed
            if to_add_time_step not in next_time_step:
                index = bisect.bisect_left(next_time_step, to_add_time_step)
                next_time_step.insert(index, to_add_time_step)
                next_jobs.insert(index, action)
            solution[action][current_time_step_job] = current_time_step
            for job in range(jobs):
                if (
                    needed_machine_jobs[job] == machine_needed
                    and legal_actions[job]
                ):
                    legal_actions[job] = False
                    nb_legal_actions -= 1
            nb_machine_legal -= 1
            machine_legal[machine_needed] = False
            for job in range(jobs):
                if illegal_actions[machine_needed][job]:
                    action_illegal_no_op[job] = False
                    illegal_actions[machine_needed][job] = False
            # if we can't allocate new job in the current timestep, we pass to the next one
            while nb_machine_legal == 0 and len(next_time_step) > 0:
                reward -= increase_time_step()
            _prioritization_non_final()
            _check_no_op()
            # we then need to scale the reward
            scaled_reward = _reward_scaler(reward)
            return (
                _get_current_state_representation(),
                scaled_reward,
                _is_done(),
                {},
            )


In [12]:
 def _reward_scaler( reward):
        return reward / max_time_op

In [13]:
def increase_time_step():
        """
        The heart of the logic his here, we need to increase every counter when we have a nope action called
        and return the time elapsed
        :return: time elapsed
        """
        hole_planning = 0
        next_time_step_to_pick = next_time_step.pop(0)
        next_jobs.pop(0)
        difference = next_time_step_to_pick -  current_time_step
        current_time_step = next_time_step_to_pick
        for job in range( jobs):
            was_left_time =  time_until_finish_current_op_jobs[job]
            if was_left_time > 0:
                performed_op_job = min(difference, was_left_time)
                time_until_finish_current_op_jobs[job] = max(
                    0,  time_until_finish_current_op_jobs[job] - difference
                )
                state[job][1] = (
                     time_until_finish_current_op_jobs[job] /  max_time_op
                )
                total_perform_op_time_jobs[job] += performed_op_job
                state[job][3] = (
                     total_perform_op_time_jobs[job] /  max_time_jobs
                )
                if  time_until_finish_current_op_jobs[job] == 0:
                     total_idle_time_jobs[job] += difference - was_left_time
                     state[job][6] =  total_idle_time_jobs[job] /  sum_op
                     idle_time_jobs_last_op[job] = difference - was_left_time
                     state[job][5] =  idle_time_jobs_last_op[job] /  sum_op
                     todo_time_step_job[job] += 1
                     state[job][2] =  todo_time_step_job[job] /  machines
                if todo_time_step_job[job] <  machines:
                         needed_machine_jobs[job] =  instance_matrix[job][
                             todo_time_step_job[job]
                        ][0]
                         state[job][4] = (
                            max(
                                0,
                                 time_until_available_machine[
                                     needed_machine_jobs[job]
                                ]
                                - difference,
                            )
                            /  max_time_op
                        )
                else:
                         needed_machine_jobs[job] = -1
                        # this allow to have 1 is job is over (not 0 because, 0 strongly indicate that the job is a
                        # good candidate)
                         state[job][4] = 1.0
                if legal_actions[job]:
                            legal_actions[job] = False
                            nb_legal_actions -= 1
            elif  todo_time_step_job[job] <  machines:
                 total_idle_time_jobs[job] += difference
                 idle_time_jobs_last_op[job] += difference
                 state[job][5] =  idle_time_jobs_last_op[job] /  sum_op
                 state[job][6] =  total_idle_time_jobs[job] /  sum_op
        for machine in range( machines):
            if  time_until_available_machine[machine] < difference:
                empty = difference -  time_until_available_machine[machine]
                hole_planning += empty
            time_until_available_machine[machine] = max(
                0,  time_until_available_machine[machine] - difference
            )
            if  time_until_available_machine[machine] == 0:
                for job in range( jobs):
                    if (
                         needed_machine_jobs[job] == machine
                        and not  legal_actions[job]
                        and not  illegal_actions[machine][job]
                    ):
                         legal_actions[job] = True
                         nb_legal_actions += 1
                    if not  machine_legal[machine]:
                             machine_legal[machine] = True
                             nb_machine_legal += 1
        return hole_planning


In [14]:
def _is_done():
    if  nb_legal_actions == 0:
        last_time_step =  current_time_step
        last_solution =  solution
        return True
    return False

In [15]:
def render(mode="human"):
        df = []
        for job in range( jobs):
            i = 0
            while i <  machines and  solution[job][i] != -1:
                dict_op = dict()
                dict_op["Task"] = "Job {}".format(job)
                start_sec =  start_timestamp +  solution[job][i]
                finish_sec = start_sec +  instance_matrix[job][i][1]
                dict_op["Start"] = datetime.datetime.fromtimestamp(start_sec)
                dict_op["Finish"] = datetime.datetime.fromtimestamp(finish_sec)
                dict_op["Resource"] = "Machine {}".format(
                     instance_matrix[job][i][0]
                )
                df.append(dict_op)
                i += 1
        fig = None
        if len(df) > 0:
            df = pd.DataFrame(df)
            fig = ff.create_gantt(
                df,
                index_col="Resource",
                colors= colors,
                show_colorbar=True,
                group_tasks=True,
            )
            fig.update_yaxes(
                autorange="reversed"
            )  # otherwise tasks are listed from the bottom up
        return fig

In [16]:
env = JssEnv()
obs = env.reset()
done = False
cum_reward = 0
while not done:
        legal_actions = obs["action_mask"]
        actions = np.random.choice(
            len(legal_actions), 1, p=(legal_actions / legal_actions.sum())
        )[0]
        obs, rewards, done, _ = env.step(actions)
        cum_reward += rewards
print(f"Cumulative reward: {cum_reward}")

NameError: name 'JssEnv' is not defined